### Load and import

In [ ]:
using Plots
using XLSX
using YAML
using CSV
using DataFrames
using Statistics
using StatsBase
using Dates

Load GSE selected data

In [ ]:
# Define the file path
general_path = @__DIR__
relative_path_GSE = "../data/GSE_prelievo_2023.xlsx"
file_GSE= joinpath(general_path,relative_path_GSE)

# Read data from the first sheet of the Excel file
GSE_profiles= DataFrame(XLSX.readtable(file_GSE,"Foglio1"));

Selection of relevant profiles

In [ ]:
# Domestic, non domestic and public lighting are selected as representative of the different sectors
# Firstly, the data is filtered to select only the column of interest
selected_columns_residential = ["PDMM",#="PDMF"=#]
selected_columns_mono = ["PDMM","PICM","PAUM"]
selected_columns_bi = ["PDMF","PICF","PAUF"]

# The date is also selected
selected_date = ["Anno","Mese","Giorno","Ora"]

selected_complete = vcat(selected_date,selected_columns_residential)

# The data is then filtered to select only the rows of interest
GSE_profiles_resident = GSE_profiles[:,selected_complete]

# Show only first 20 rows
#first(GSE_profiles_resident,15)

# Relative or absolute values

In [ ]:
#= ARERA_all_power_consumption = [202, 168, 183, 154, 151, 174, 215, 179, 148, 142, 158, 176]
ARERA_1_3_power_consumption = [174, 144, 151, 133, 128, 143, 180, 158, 125, 122, 133, 152]
ARERA_average_month = [mean(ARERA_all_power_consumption) for i in 1:12] =#
Antrodoco_profiles = [189, 162, 212, 196, 184, 138, 148, 154, 145, 171, 195, 206]

# The data is then multiplied or not by ARERA values
# Select "Relative" or "Absolute" to choose the type of data
status = "Absolute"

# The GSE data is multiplied by the ARERA data to obtain the total consumption
# For each monthly firstly we do the sum over PDMM and then multiply by the ARERA data
GSE_monthly_sum = zeros(12)
if status == "Absolute"
    for i in 1:12
        GSE_monthly_sum[i] = sum(GSE_profiles_resident[GSE_profiles_resident[:,:Mese].==i,:PDMM])
        # The GSE data is then multiplied by the ARERA data when the month is the same
        GSE_profiles_resident[GSE_profiles_resident.Mese .== i,:PDMM] = GSE_profiles_resident[GSE_profiles_resident.Mese .== i,:PDMM] .* Antrodoco_profiles[i] ./ GSE_monthly_sum[i]
    end
end

# Plots

In [ ]:
# Assumendo che "Ora" sia una colonna in GSE_profiles e che tu voglia calcolare la media di una colonna chiamata "Valore"
grouped = groupby(GSE_profiles_resident, :Ora)
average_curve = combine(grouped, :PDMM => mean => :Valore_medio)
median_curve = combine(grouped, :PDMM => median => :Valore_mediano)
maximum_curve = combine(grouped, :PDMM => maximum => :Valore_massimo)
minimum_curve = combine(grouped, :PDMM => minimum => :Valore_minimo)

# Create base plot
if status == "Absolute"
    p0 = plot(xlab = "Ora", ylab = "Potenza [kW]",)
else
    p0 = plot(xlab = "Ora", ylab = "Distribuzione [%]",)
end

p1 = deepcopy(p0)


# Plot the data
plot!(p1, average_curve.Ora, average_curve.Valore_medio, label = "Valore medio", color = :red)
plot!(p1, median_curve.Ora, median_curve.Valore_mediano, label = "Valore mediano", color = :red, line = :dash)
plot!(p1, maximum_curve.Ora, maximum_curve.Valore_massimo, label = "Valore massimo", color = :blue)
plot!(p1, minimum_curve.Ora, minimum_curve.Valore_minimo, label = "Valore minimo", color = :green)

# Show the plot
display(p1)

# Save the plot
savefig(joinpath(general_path,"../results/Immagini/GSE/GSE_statistics.png"))

In [ ]:
# Raggruppa i dati per :Ora e :Mese e calcola la media
grouped_hourly = groupby(GSE_profiles_resident, [:Ora, :Mese])
average_curve_hourly = combine(grouped_hourly, :PDMM => mean => :Valore_medio)

# Crea un nuovo plot
p2 = deepcopy(p0)

# Plotta i dati
for i in unique(average_curve_hourly.Mese)
    subset = average_curve_hourly[average_curve_hourly.Mese .== i, :]
    plot!(p2, subset.Ora, subset.Valore_medio, label = "Mese $i", color = rand(RGB))
end

# Mostra il plot
display(p2)

# Salva il plot
savefig(joinpath(general_path,"../results/Immagini/GSE/GSE_all_months.png"))


In [ ]:
# Mostrare la varianza per ogni ora sull'asse secondario y
# Raggruppa i dati per :Ora e calcola la varianza e la covarianza
grouped_hourly = groupby(GSE_profiles_resident, [:Ora])
average_curve_hourly = combine(grouped_hourly, :PDMM => mean => :Valore_medio)
variance_curve_hourly = combine(grouped_hourly, :PDMM => var => :Varianza)


# Crea un nuovo plot e aggiungi l'asse secondario y
p2_1 = deepcopy(p0)

# Plotta i dati ma l'asse secondario y è utilizzato per la varianza e la covarianza a 10^-3
plot!(p2_1, average_curve_hourly.Ora, average_curve_hourly.Valore_medio, label = "Valore medio", color = :red)
p_secondary = twinx()
plot!(p_secondary, variance_curve_hourly.Ora, variance_curve_hourly.Varianza, label = "Varianza", color = :blue, legend = :top, ylabel = "Varianza [kW^2]")

# Mostra il plot
display(p2_1)

# Salva il plot
savefig(joinpath(general_path,"../results/Immagini/GSE/GSE_var.png"))

In [ ]:
# Definisci una funzione per ottenere l'indice del valore massimo, minimo e medio
function get_indices(df)
    max_index = argmax(df.PDMM)
    min_index = argmin(df.PDMM)
    mean_value = mean(df.PDMM)
    _, median_index = findmin(abs.(df.PDMM .- median(df.PDMM)))
    _, mean_index = findmin(abs.(df.PDMM .- mean_value))
    return (max_index = df[max_index, :Mese], min_index = df[min_index, :Mese], mean_index = df[mean_index, :Mese], median_index = df[median_index, :Mese])
end

# Applica la funzione a ciascun gruppo
indices = combine(groupby(GSE_profiles_resident, :Ora), get_indices)

# Conta le ricorrenze dei valori in ciascuna colonna
max_counts = sort(countmap(indices.max_index), rev=false)
min_counts = sort(countmap(indices.min_index), rev=false)
mean_counts = sort(countmap(indices.mean_index), rev=false)
median_counts = sort(countmap(indices.median_index), rev=false)

# Stampa i risultati
println("Max counts: ", max_counts)
println("Min counts: ", min_counts)
println("Mean counts: ", mean_counts)
println("Median counts: ", median_counts)

In [ ]:
#Aggiungo una nuova colonna che calcola il giorno della settimana utilizzando le colonne anno, mese e giorno
GSE_profiles_resident[!,:Giorno_settimana] = map(row -> Dates.dayname(Dates.Date(row[:Anno], row[:Mese], row[:Giorno])), eachrow(GSE_profiles_resident))

# Voglio ottenere la media della colonna PDMM per ciascun giorno della settimana
grouped_weekday = groupby(GSE_profiles_resident, [:Giorno_settimana,:Ora])
average_curve_weekday = combine(grouped_weekday, :PDMM => mean => :Valore_medio)

# Crea un nuovo plot
p3 = deepcopy(p0)

# Plotta i dati
for i in unique(average_curve_weekday.Giorno_settimana)
    subset = average_curve_weekday[average_curve_weekday.Giorno_settimana .== i, :]
    plot!(p3, subset.Ora, subset.Valore_medio, label = i, color = rand(RGB))
end

# Mostra il plot
display(p3)

# Salva il plot
savefig(joinpath(general_path,"../results/Immagini/GSE/GSE_weekday.png"))

# Export results

In [ ]:
# Save the data as a csv file for further analysis
CSV.write(joinpath(general_path,"../results/Dati/GSE_profiles_resident_v1_Antrodoco.csv"), GSE_profiles_resident)